In [ ]:
%load_ext autoreload
%autoreload 2
%run -n main.py

# datasets

In [ ]:
# path = join_path(DATA_DIR, DATASET)
# !mkdir -p {path}

In [ ]:
# for name in DATASETS:
#     file = CORUS_FILES[name]
#     if name == AE:
#         path = [join_path(CORUS_DATA_DIR, _) for _ in file]
#     else:
#         path = join_path(CORUS_DATA_DIR, file)
    
#     records = DATASETS[name](path)
#     records = log_progress(records, desc=name)
#     items = as_jsons(records)
#     lines = format_jl(items)
#     path = join_path(DATA_DIR, DATASET, name + JL + GZ)
#     dump_gz_lines(lines, path)

In [ ]:
datasets = {}
for name in log_progress(DATASETS):
    path = join_path(DATA_DIR, DATASET, name + JL + GZ)
    lines = load_gz_lines(path)
    items = parse_jl(lines)
    datasets[name] = list(from_jsons(items, Sim))

# models

In [ ]:
# for scheme in SCHEMES:
#     path = join_path(DATA_DIR, scheme.name)
#     !mkdir -p {path}

In [ ]:
# dir = RUSVEC_DATA_DIR
# # !wget http://vectors.nlpl.eu/repository/11/180.zip -O {dir}/ruscorpora_upos_cbow_300_20_2019.zip
# # !unzip {dir}/ruscorpora_upos_cbow_300_20_2019.zip -d {dir}/ruscorpora_upos_cbow_300_20_2019
# # !rm {dir}/ruscorpora_upos_cbow_300_20_2019.zip {dir}/ruscorpora_upos_cbow_300_20_2019/model.txt

# # !wget http://vectors.nlpl.eu/repository/11/182.zip -O {dir}/ruwikiruscorpora_upos_skipgram_300_2_2019.zip
# # !unzip {dir}/ruwikiruscorpora_upos_skipgram_300_2_2019.zip -d {dir}/ruwikiruscorpora_upos_skipgram_300_2_2019
# # !rm {dir}/ruwikiruscorpora_upos_skipgram_300_2_2019.zip {dir}/ruwikiruscorpora_upos_skipgram_300_2_2019/model.txt

# # !wget http://vectors.nlpl.eu/repository/11/185.zip -O {dir}/tayga_upos_skipgram_300_2_2019.zip
# # !unzip {dir}/tayga_upos_skipgram_300_2_2019.zip -d {dir}/tayga_upos_skipgram_300_2_2019
# # !rm {dir}/tayga_upos_skipgram_300_2_2019.zip {dir}/tayga_upos_skipgram_300_2_2019/model.txt

# # !wget http://vectors.nlpl.eu/repository/11/187.zip -O {dir}/tayga_none_fasttextcbow_300_10_2019.zip
# # !unzip {dir}/tayga_none_fasttextcbow_300_10_2019.zip -d {dir}/tayga_none_fasttextcbow_300_10_2019
# # !rm {dir}/tayga_none_fasttextcbow_300_10_2019.zip

# # !wget https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz -O {dir}/araneum_none_fasttextcbow_300_5_2018.tgz
# # !mkdir {dir}/araneum_none_fasttextcbow_300_5_2018
# # !tar xzvf {dir}/araneum_none_fasttextcbow_300_5_2018.tgz -C {dir}/araneum_none_fasttextcbow_300_5_2018
# # !rm {dir}/araneum_none_fasttextcbow_300_5_2018.tgz

In [ ]:
# for scheme in SCHEMES:
#     log(scheme.name)
#     model = scheme.load()
#     for dataset_name in datasets:
#         records = datasets[dataset_name]
#         records = log_progress(records, desc=dataset_name)
#         records = map_model(model, records, scheme.tagged)
#         items = as_jsons(records)
#         lines = format_jl(items)
#         path = join_path(DATA_DIR, scheme.name, dataset_name + JL + GZ)
#         dump_gz_lines(lines, path)
        
#         records = datasets[dataset_name]
#         for record in records[:1000]:  # measure get perf
#             model.get(record.a)
            
#         path = join_path(DATA_DIR, scheme.name, STATS + JSON)
#         text = format_json(scheme.stats.as_json)
#         dump_text(text, path)

# score

In [ ]:
stats = {}
for scheme in SCHEMES:
    path = join_path(DATA_DIR, scheme.name, STATS + JSON)
    text = load_text(path)
    data = parse_json(text)
    stats[scheme.name] = Stats.from_json(data)

In [ ]:
scores = {}
keys = [
    (scheme.name, dataset)
    for scheme in SCHEMES
    for dataset in DATASETS
]
for model, dataset in log_progress(keys):
    targets = datasets[dataset]

    path = join_path(DATA_DIR, model, dataset + JL + GZ)
    lines = load_gz_lines(path)
    items = parse_jl(lines)
    preds = list(from_jsons(items, Sim))

    score = SCORES[dataset](preds, targets)
    scores[model, dataset] = score

# report

In [ ]:
table = report_table(scores, stats, DATASETS, SCHEMES)
html = format_report(table, DATASETS)
HTML(html)

In [ ]:
html = format_github_report1(table)
patch_readme(EMB1, html, README)
patch_readme(EMB1, html, NAVEC_README)
HTML(html)

In [ ]:
html = format_github_report2(table, DATASETS)
patch_readme(EMB2, html, README)
patch_readme(EMB2, html, NAVEC_README)
HTML(html)